In [952]:
import csv
import requests
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from datetime import datetime
import matplotlib.patches as patches
import matplotlib.lines as lines

plt.close('all')

In [953]:
CSV_URL = 'https://covidtracking.com/api/v1/states/daily.csv'

with requests.Session() as s:
    download = s.get(CSV_URL)
    download
    decoded_content = download.content.decode('utf-8')
    cr = pd.read_csv(StringIO(decoded_content))
    df = pd.DataFrame(cr)   

In [954]:
date = pd.to_datetime(df['date'].astype(str), format='%Y%m%d')
df = pd.DataFrame({'date':date, 'state':df['state'], 'positiveIncrease':df['positiveIncrease']})

stateNames = ['WA', 'ID', 'MT', 'ND', 'MN','WI','MI','NY','VT', 'NH','ME',
             'OR','WY', 'SD', 'IA', 'IL', 'IN', 'OH', 'PA', 'MA', 'RI', 'CT', 'NJ',
             'DE', 'MD', 'DC', 'CA', 'NV', 'UT', 'CO', 'NE', 'MO', 'KS', 'OK', 'NM', 
              'AZ', 'TX', 'LA', 'AR', 'TN', 'MS', 'AL', 'KY', 'WV', 'VA', 'NC',
             'SC', 'GA', 'FL', 'AK', 'HI']
dfstates = []
len(stateNames)

51

In [955]:
# I'm sure there's a cleaner way to do this...

for name in stateNames:
    sel = df[(df['state']==name)]
    sel.index=sel['date']
    dfstates.insert(len(dfstates), sel)
s = 'positiveIncrease'
dfgroup = pd.DataFrame()

for d in dfstates:
    dfgroup[d['state'][0]] = d[s]

In [956]:
#Define color pallete

redBar = '#ECCFC2'
twoWeeksRisingColor = '#F0EBD8'
threeWeeksIncreaseColor = '#F2DFAE'
sevenDayDecreaseColor = '#D8F0DF'
borderColor = "#E1DDBF"
fourWeeksIncreaseColor = '#F5DFDC'

In [957]:
def formatBorder(ax):
    for spineName in ax.spines:
        ax.spines[spineName].set_color(borderColor)
        ax.spines[spineName].set_linewidth(2)

In [958]:
def sumRange(L,a,b):                                                                                                                                                                                                
    sum = 0                                                                                                                                                                                                         
    for i in range(a,b+1,1):                                                                                                                                                                                        
        sum += L[i]                                                                                                                                                                                                  
    return sum      

In [959]:
def consecutiveWeeksIncreased(state):
    weeks = 0
    prev = 999999
    for week in range(10):
        thisWeekSum = sumRange(state, weeks * 7, weeks * 7 + 8)
        if thisWeekSum <= prev:
            prev = thisWeekSum
            weeks += 1
        else:
            return weeks
    return weeks

In [960]:
state = dfgroup['WA']
week = 0
weekSum = sumRange(state, week * 7, week * 7 + 8)
weekSum

4909

In [961]:
week = 1
weekSum = sumRange(state, week * 7, week * 7 + 8)
weekSum

2280

In [978]:
def do_plot(ax, name):
    color = 'C'+str(dfgroup.columns.get_loc(name))
    ax.plot(dfgroup[name], color=(color), linewidth=1)
    ax.set_xlabel('')
    ax.grid(False)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    ax.set_title(name,horizontalalignment='left', color=(color), fontweight="bold", fontsize=12, x=0.025, y=.75)
    formatBorder(ax)
    weeksClimbing = consecutiveWeeksIncreased(dfgroup[name])
    if(weeksClimbing >= 4):
        ax.set_facecolor(fourWeeksIncreaseColor)
    else :
        if(weeksClimbing >=3):
            ax.set_facecolor(threeWeeksIncreaseColor)
        else:
            if(weeksClimbing >= 2):
                ax.set_facecolor(twoWeeksRisingColor)
    ax.axhline(y=100, color='r', linestyle='-', linewidth=.2)
    #ax.annotate(s= str(weeksClimbing), xy=(0.1, 0.1), xycoords='axes fraction')

In [979]:
def add_state(figure, stateName, grid):
    state = figure.add_subplot(grid)
    do_plot(state, stateName)
    return state

In [980]:
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
dt_string = "Last updated " + dt_string

In [981]:
def add_column(fig, names, column, grid):
    row = 0
    for name in names:
        if(name == ""):
            row+=1
            continue
        else:
            add_state(fig, name, grid[row, column])
            row+=1

In [ ]:
plt.style.use('fivethirtyeight')

g = gridspec.GridSpec(11, 11)

fig = plt.figure(figsize=(25,20))
titleColor = '#706343'
labelColor = '#706D65'
fig.suptitle('US Daily COVID-19 Case Increases', y=0.85, fontsize=24, color=titleColor)
fig.text(x=0, y=.25, s=dt_string, color = labelColor)

x = .1
y =.75


#Create legend
redline = mlines.Line2D([], [], color=redBar, marker='', markersize=15, label='100 new cases per day')
twoWeeksIncrease = mlines.Line2D([],[], color = twoWeeksRisingColor, marker='s',markersize=30, label='Rising for 2 weeks')
threeWeeksIncrease = mlines.Line2D([],[], color = threeWeeksIncreaseColor, marker='s',markersize=30, label='Rising for 3 weeks')
fourWeeksIncrease = mlines.Line2D([],[], color = fourWeeksIncreaseColor, marker='s',markersize=30, label='Rising for 4+ weeks')

legend = fig.legend(handles=[redline, twoWeeksIncrease, threeWeeksIncrease, fourWeeksIncrease], loc='upper left',borderaxespad=7, framealpha=0, labelspacing=2)

for t in legend.get_texts():
    t.set_color(labelColor)
    

# Create state subplots
add_column(fig, ['', '', 'WA', 'OR', 'CA',  '', '', 'HI'], 0, g)
add_column(fig, ['', '', 'ID', 'NV', 'UT', 'AZ', '', 'AK'], 1, g)
add_column(fig, ['','', 'MT', 'WY', 'CO', 'NM'], 2, g)
add_column(fig, ['','','ND','SD', 'NE', 'KS', 'OK', 'TX'], 3, g)
add_column(fig, ['', '', 'MN', 'IA', 'MO', 'AR', 'LA'], 4, g)
add_column(fig, ['', 'WI', 'IL', 'IN', 'KY', 'TN', 'MS'], 5, g)
add_column(fig, ['', '','MI', 'OH', 'WV', 'NC', 'AL'], 6, g)
add_column(fig, ['', '', '', 'PA', 'VA', 'SC', 'GA'], 7, g)
add_column(fig, ['', '', 'NY', 'NJ', 'MD', '', '', 'FL'], 8, g)
add_column(fig, ['', 'VT', 'MA', 'CT', 'DE'], 9, g)
add_column(fig, ['ME', 'NH', 'RI', 'DC'], 10, g)

#Data source label
fig.text(x=.7, y=.25, s='Data collected from covidtracking.com', color = labelColor, horizontalAlignment='right')